# Local Education Agency Finance Data: Cleaning & Automation Process Preparation
Local Education Agency will be abbreviated as LEA

## Data Source and Selection Criteria

**Data Source:**
"Common Core of Data School District Finance Survey (F-33), FY 2010 – 2020," provided by the National Center for Education Statistics, U.S. Department of Education. 

Accessed from: 
- [NCES Website – CCD Data Files](https://nces.ed.gov/ccd/files.asp#Fiscal:1,Page:1)

**Filters Applied on CCD Data Files Search:**
- Fiscal/Nonfiscal: Fiscal
- Level: District
- School Year (2010-2020):
    - 2019-2020
    - 2018-2019
    - 2017-2018
    - 2016-2017
    - 2015-2016
    - 2014-2015
    - 2013-2014
    - 2012-2013
    - 2011-2012
    - 2010-2011

**Files Downloaded:**
- Data File: Flat File  (.zip containing .txt format)
- Documentation:
    - General Documentation (when available)
    - Record Layout (used in lieu of General Documentation when not available)  
    

Note:  
This notebook served as a crucial tool during the initial stages of the project, primarily focusing on the exploratory analysis of educational finance data. My journey began with the LEA School District Finance Survey data for the year 2020. I methodically worked backwards through historical data, rigorously testing the code with each year's files. This systematic approach culminated in the creation of the "LEA Local Finance Survey – School District Data 2010 – 2020 – Column Mapping.xlsx" file.

The purpose of this Excel file was twofold: firstly, to document and map the evolution of column names across different years, and secondly, to provide clear definitions for each data point. These definitions were derived from the General Documentation provided with the raw data files by the National Center for Education Statistics (NCES). 

One of the key innovations in my approach was the dynamic renaming of columns using this Excel file. This strategy eliminated the need to hardcode column names into the script, significantly enhancing the flexibility and scalability of my data processing method. 

After thorough testing and refinement of the mapping process within this notebook, the code was then successfully exported and adapted into a Python (.py) file. This adaptation was a pivotal step towards automating the insertion of a decade's worth of data into my database. 

I decided to keep this notebook as part of the repository. It serves an invaluable role in elucidating the data cleaning and preparation steps for external audiences who might not be familiar with the intricacies of the code. This notebook stands as a testament to my meticulous approach and commitment to clarity and transparency in data processing.

## Import Packages

In [37]:
import pandas as pd
import numpy as np
import os
import json
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## Importing the Dataset

When importing the dataset, follow these steps for best practices and to ensure accuracy:

1. **Locate the Dataset:**
   Ensure that the dataset file is present in the project directory. This is where the import function will look for the file.

2. **Understand the File Format:**
   The dataset is in a TAB-delimited format. When using `pandas.read_csv` or similar functions, specify the delimiter with `delimiter='\t'` to correctly parse the file.

3. **Verify the Import:**
   After importing, it's crucial to do a quick check of the DataFrame:
   - Use `df.head()` to preview the first few rows.


I renamed the original files names to be easier to discern when reading them into the notebook. 

In [38]:
file_name = 'sdf17.txt'
file_path = os.path.join("..", "raw_data_files", file_name)

df = pd.read_csv(file_path, delimiter= '\t')

df.head()

/var/folders/y0/k7d2_xkj0mj474bsznv2mc6r0000gn/T/ipykernel_18601/3443625066.py:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter= '\t')


,LEAID,CENSUSID,FIPST,CONUM,CSA,CBSA,NAME,STNAME,STABBR,SCHLEV,...,FL_61V,FL_66V,FL_W01,FL_W31,FL_W61,FL_V95,FL_V02,FL_K14,FL_CE1,FL_CE2
0,100002,N,1,1073,142,13820,Alabama Youth Services,Alabama,AL,N,...,M,M,M,M,M,M,M,M,M,M
1,100005,01504840100000,1,1095,290,10700,Albertville City,Alabama,AL,03,...,M,M,R,R,R,R,M,M,M,M
2,100006,01504800100000,1,1095,290,10700,Marshall County,Alabama,AL,03,...,M,M,R,R,R,R,M,M,M,M
3,100007,01503740100000,1,1073,142,13820,Hoover City,Alabama,AL,03,...,M,M,R,R,R,R,M,M,M,M
4,100008,01504530100000,1,1089,290,26620,Madison City,Alabama,AL,03,...,M,M,R,R,R,R,M,M,M,M


## Import the Column Mapping
I prepared an excel file that has the original column names, the new names of the columns, their expected datatype in a database, and the description. This file will serve as a quick and easy way to map the new columns with less code, and maintaining a dictionary of the columns.

In [39]:
# Import Column Map
file_name = 'LEA Local Finance Survey – School District Data 2010 – 2020 – Column Mapping.xlsx'
column_map_path = os.path.join("..", file_name)

column_mapping_df = pd.read_excel(column_map_path, sheet_name='Column Mapping 16')

# Remove White Spaces from Column Names
column_mapping_df['Original Name'] = column_mapping_df['Original Name'].str.strip()
column_mapping_df['New Name'] = column_mapping_df['New Name'].str.strip()

column_mapping_df.head()

,Original Name,New Name,Type,Table,Description
0,LEAID,lea_id,VARCHAR(7),entity,National Center For Education Statistics (NCES...
1,CENSUSID,census_id,VARCHAR(14),all,Census Bureau 14-Digit Government Id
2,FIPST,ansi_state_code,VARCHAR(2),entity,American National Standards Institute (ANSI) S...
3,CONUM,ansi_county_code,VARCHAR(7),entity,American National Standards Institute (ANSI) C...
4,CSA,csa,VARCHAR(3),entity,Consolidated Statistical Area


In [40]:
# Create Dictionary to Map New Column Names
column_map_dict = column_mapping_df.set_index('Original Name')['New Name'].to_dict()

# Rename Columns
df.rename(columns=column_map_dict, inplace=True)
df.columns = df.columns.str.strip()

In [41]:
df.dtypes

lea_id                                      object
census_id                                   object
ansi_state_code                              int64
ansi_county_code                            object
csa                                         object
                                             ...  
utilities_services_flag                     object
tech_related_supplies_services_flag         object
tech_related_equipment_flag                 object
curr_expenditures_state_local_funds_flag    object
curr_expenditures_federal_funds_flag        object
Length: 260, dtype: object

After renaming the columns, I used `df.dtypes` to confirm the names of the columns were correct, but also so I can get an idea of what columns may need cleaning to acheive a certain data type.

## Exclusion of Non-Government Entities from Analysis

The Census Bureau has specific criteria to determine if a Local Education Agency (LEA) qualifies as a government entity. These criteria include the LEA's power to:

- Levy taxes
- Independently manage its own budget
- Appoint its school board members without oversight from other local government bodies

A LEA that satisfies these conditions is considered a government entity and is assigned a unique `census_id`. This identifier signals eligibility for federal, state, and local funding, which is often dependent on an LEA's tax authority and fiscal independence.

However, LEAs that do not meet these criteria are assigned an 'N' for their `census_id`. This indicates that they are not recognized as government entities by the Census Bureau and, consequently, are not typically eligible for the tax-based funding that our analysis focuses on. Therefore, these LEAs are excluded from the dataset to maintain a focus on entities eligible for such funding.

By removing rows where `census_id` is 'N', I ensure that the analysis only includes LEAs that have the potential to receive and manage federal, state, and local funding in line with my research objectives.


In [42]:
# Remove rows where census_id is N
df = df[df['census_id'] != 'N']

In [43]:
# Check for duplicated values in census_id
df['census_id'].duplicated().any()

False

After removing the Census IDs that had the 'N' placeholder, I wanted to confirm that there were no duplicate Census IDs. This is in preparation for this being the Primary Key within the database table keeping the LEA Entity information. This will serve as a Foreign Key in subsequent tables to link records to the Entity.  
#### Expected Result : False

## Column Removal for Database Normalization

As part of the data normalization process for database insertion, I target columns starting with 'total_' for removal. These columns are presumed to contain aggregate data that may not be suitable for the normalized database structure. Prior to their removal, the content of these columns is preserved by transferring it to a separate DataFrame. This precaution ensures that the aggregate data remains accessible for any future analysis or reference requirements.

In [44]:
total_columns = []

for col in df.columns: 
    if col.startswith('total_'):
        total_columns.append(col)

total_columns

['total_revenue',
 'total_federal_revenue',
 'total_state_revenue',
 'total_local_revenue',
 'total_expenditures',
 'total_curr_expenditures_pri_sec_ed',
 'total_curr_expenditures_instruction',
 'total_curr_expenditures_support_services',
 'total_current_expenditures_other_prim_sec',
 'total_non_prim_sec_expenditures',
 'total_capital_outlay_expenditures',
 'total_salaries',
 'total_employee_benefits',
 'total_salaries_flag',
 'total_employee_benefits_flag']

In [45]:
column_totals = df[total_columns].copy()
column_totals

,total_revenue,total_federal_revenue,total_state_revenue,total_local_revenue,total_expenditures,total_curr_expenditures_pri_sec_ed,total_curr_expenditures_instruction,total_curr_expenditures_support_services,total_current_expenditures_other_prim_sec,total_non_prim_sec_expenditures,total_capital_outlay_expenditures,total_salaries,total_employee_benefits,total_salaries_flag,total_employee_benefits_flag
1,52593000,7049000,31601000,13943000,50456000,45298000,25785000,15790000,3723000,849000,2496000,24804000,10088000,R,R
2,56550000,6879000,36160000,13511000,59341000,54735000,28419000,22178000,4138000,753000,3128000,30829000,12436000,R,R
3,195251000,6727000,72828000,115696000,165480000,149911000,90762000,51164000,7985000,3275000,4799000,88502000,34715000,R,R
4,110565000,5171000,59814000,45580000,105184000,97512000,58769000,34007000,4736000,1186000,1937000,54983000,21085000,R,R
6,21345000,1575000,11516000,8254000,22552000,18974000,10378000,7523000,1073000,287000,806000,10319000,4006000,R,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18671,57295000,2321000,6217000,48757000,54387000,50406000,31105000,17961000,1340000,0,3553000,29872000,14063000,R,R
18672,7110000,200000,5445000,1465000,6872000,6240000,3419000,2616000,205000,67000,565000,3480000,1627000,R,R
18673,25969000,1793000,18805000,5371000,27122000,23090000,14860000,7423000,807000,0,3730000,12897000,6048000,R,R
18674,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,N,N


In [46]:
df.drop(columns= total_columns, inplace= True)

In [47]:
df['year'] = df['year'].astype(str)
df['year'] = '20' + df['year']
df['year']

1        2017
2        2017
3        2017
4        2017
6        2017
         ... 
18671    2017
18672    2017
18673    2017
18674    2017
18675    2017
Name: year, Length: 14667, dtype: object

## Casting Data Types
In the above cell, I am converting year to a String so I can add '20' to the year in order to have the correct format to convert to datetime.
In the below cells:
- ansi_state_code and ansi_county_code are converted to Strings because these will not be aggregated at any point. 
- year is being converted to datetime.
- ccd_nonfiscal_match and census_fiscal_match are being converted to booleans to match database data type requirements.

In [48]:
df['ansi_state_code'] = df['ansi_state_code'].astype(str)
df['ansi_county_code'] = df['ansi_county_code'].astype(str)
df['year'] = pd.to_datetime(df['year'].astype(str), format='%Y')
df['ccd_nonfiscal_match'] = df['ccd_nonfiscal_match'].astype(bool)
df['census_fiscal_match'] = df['census_fiscal_match'].astype(bool)

In [49]:
df[['ansi_state_code', 'ansi_county_code', 'year', 'ccd_nonfiscal_match', 'census_fiscal_match']].dtypes

ansi_state_code                object
ansi_county_code               object
year                   datetime64[ns]
ccd_nonfiscal_match              bool
census_fiscal_match              bool
dtype: object

In [50]:
df.describe(include='all')

,lea_id,census_id,ansi_state_code,ansi_county_code,csa,cbsa,lea_name,state,st_abbr,school_level_code,...,short_term_debt_outstanding_begin_fisc_year_flag,short_term_debt_outstanding_end_fisc_year_flag,assets_sinking_fund_flag,assets_bond_fund_flag,assets_other_funds_flag,utilities_services_flag,tech_related_supplies_services_flag,tech_related_equipment_flag,curr_expenditures_state_local_funds_flag,curr_expenditures_federal_funds_flag
count,14667.0,14667,14667,14667,14667,14667,14667,14667,14667,14667,...,14667,14667,14667,14667,14667,14667,14667,14667,14667,14667
unique,14667.0,14667,51,3182,174,932,14351,51,51,7,...,5,4,5,5,5,4,4,4,5,5
top,100005.0,01504840100000,6,17031,N,N,Jefferson County,California,CA,03,...,R,R,R,R,R,R,M,R,M,M
freq,1.0,1,1123,165,6987,3818,5,1123,1123,10458,...,11049,11052,13151,13234,13563,8853,7351,7208,7193,7193
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning Notes

**Handling Special Placeholders in Financial Data:**

The dataset uses special placeholder values to indicate non-standard entries for financial data: 
- “-1” indicates missing data, which may arise in situations where zero values are ambiguous.
- “-2” and “-3” could similarly indicate other forms of non-standard or suppressed data, such as revised figures or privacy-related omissions.

To facilitate accurate analysis, I replaced these placeholder values in the money-related fields to avoid distortions in statistical calculations. However, each financial field is paired with a corresponding "flag" column. These flag columns provide references to documentation that explain the classification of each value in more depth, including the placeholders.

The purpose of this cleaning step is not to discard the nuances and details encoded by these placeholders but to create a dataset that can be analyzed quantitatively without misinterpretation caused by non-numeric values. The flag columns remain intact for any case-by-case examination where the context behind the numeric values is necessary, ensuring transparency and traceability in the dataset.

This approach ensures that while the dataset is primed for quantitative analysis, the integrity and comprehensiveness of the data are maintained for more qualitative assessments.


In [51]:
df.replace([-9, -3, -2, -1], np.nan, inplace=True)

In [52]:
df.describe()

,year,fall_membership,fall_membership_school_univ,title_I_thru_state,indiv_with_disabilities_thru_state,math_science_teacher_quality_thru_state,national_activities_for_school_safety_thru_state,voc_tech_education_thru_state,bilingual_education_thru_state,other_thru_state,...,short_term_debt_outstanding_end_fisc_year,assets_sinking_fund,assets_bond_fund,assets_other_funds,utilities_services,tech_related_supplies_services,tech_related_equipment,curr_expenditures_state_local_funds,curr_expenditures_federal_funds,weight
count,14667,13413.000000,13688.000000,1.430600e+04,1.430600e+04,1.430600e+04,1.430600e+04,1.430600e+04,1.430600e+04,1.430600e+04,...,1.430600e+04,1.430600e+04,1.430600e+04,1.430600e+04,1.430600e+04,1.430600e+04,1.430600e+04,7.160000e+03,7.160000e+03,14667.0
mean,2017-01-01 00:00:00,3625.271677,3538.894287,9.864321e+05,7.889962e+05,1.001259e+05,4.575073e+03,3.854823e+04,2.345303e+04,4.065243e+05,...,6.429395e+05,1.538295e+06,4.964130e+06,1.110739e+07,5.593697e+05,2.446283e+05,7.524640e+04,3.390240e+07,3.316881e+06,1.0
min,2017-01-01 00:00:00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0
25%,2017-01-01 00:00:00,405.000000,376.000000,5.800000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,1.071000e+06,0.000000e+00,0.000000e+00,0.000000e+00,4.250250e+06,3.020000e+05,1.0
50%,2017-01-01 00:00:00,1110.000000,1064.000000,1.890000e+05,1.360000e+05,1.100000e+04,0.000000e+00,0.000000e+00,0.000000e+00,4.200000e+04,...,0.000000e+00,1.000000e+03,0.000000e+00,3.249500e+06,8.000000e+04,0.000000e+00,0.000000e+00,1.033200e+07,7.515000e+05,1.0
75%,2017-01-01 00:00:00,2971.000000,2887.250000,5.670000e+05,5.630000e+05,6.400000e+04,0.000000e+00,1.400000e+04,0.000000e+00,1.960000e+05,...,0.000000e+00,5.610000e+05,7.787500e+05,9.028750e+06,3.970000e+05,7.475000e+04,0.000000e+00,2.600050e+07,2.087500e+06,1.0
max,2017-01-01 00:00:00,984462.000000,969230.000000,9.568510e+08,2.495270e+08,4.517300e+07,2.617000e+06,1.131400e+07,1.578000e+07,1.626490e+08,...,1.336994e+09,8.562310e+08,1.348351e+09,3.398387e+09,7.844700e+07,1.605370e+08,2.497600e+07,2.852557e+09,4.484950e+08,1.0
std,NaN,14855.812715,14618.899791,9.633560e+06,3.733459e+06,6.464771e+05,5.837162e+04,1.984524e+05,2.071620e+05,2.469664e+06,...,1.199405e+07,1.124702e+07,2.822301e+07,4.248822e+07,2.299994e+06,1.951852e+06,5.326310e+05,1.142238e+08,1.377555e+07,0.0


### Entity Schema Tables  

#### Create entity DataFrame

In [53]:
# Initialize an empty list for storing column names
entity_columns = []

# Iterate over each row in the mapping DataFrame
for index, row in column_mapping_df.iterrows():
    # Check if the table is 'entity' or 'all', and the column name is not 'year'
    if row['Table'] in ['entity', 'all'] and row['New Name'] != 'year':
        if row['New Name'] not in total_columns:
            # Add the new column name to the list
            entity_columns.append(row['New Name'])

# Create a new DataFrame with only the selected columns
entity = df[entity_columns].copy()
entity


,lea_id,census_id,ansi_state_code,ansi_county_code,csa,cbsa,lea_name,state,st_abbr
1,100005,01504840100000,1,1095,290,10700,Albertville City,Alabama,AL
2,100006,01504800100000,1,1095,290,10700,Marshall County,Alabama,AL
3,100007,01503740100000,1,1073,142,13820,Hoover City,Alabama,AL
4,100008,01504530100000,1,1089,290,26620,Madison City,Alabama,AL
6,100011,01503710100000,1,1073,142,13820,Leeds City,Alabama,AL
...,...,...,...,...,...,...,...,...,...
18671,5605830,51502000200000,56,56039,N,27220,Teton County School District #1,Wyoming,WY
18672,5606090,51502300200000,56,56045,N,N,Weston County School District #7,Wyoming,WY
18673,5606240,51502200400000,56,56043,N,N,Washakie County School District #1,Wyoming,WY
18674,5680180,51500340100000,56,56005,N,23940,Northeast Wyoming BOCES,Wyoming,WY


#### Create annual_stats DataFrame

In [54]:
# Initialize an empty list for storing column names
annual_stats_columns = []

# Iterate over each row in the mapping DataFrame
for index, row in column_mapping_df.iterrows():
    # Check if the table is 'annual_stats' or 'all'
    if row['Table'] in ['annual_stats', 'all']:
        if row['New Name'] not in total_columns:
            # Add the new column name to the list
            annual_stats_columns.append(row['New Name'])

# Create a new DataFrame with only the selected columns
annual_stats = df[annual_stats_columns].copy()

# If 'year' is not the last column, move it to the end
if 'year' in annual_stats.columns and annual_stats.columns[-1] != 'year':
    # Get a list of all columns except 'year'
    cols = [col for col in annual_stats.columns if col != 'year']
    # Add 'year' at the end of the list
    cols.append('year')
    # Reorder the DataFrame
    annual_stats = annual_stats[cols]

annual_stats

,census_id,school_level_code,agency_charter_code,ccd_nonfiscal_match,census_fiscal_match,low_grade_offered,high_grade_offered,fall_membership,fall_membership_school_univ,fall_membership_flag,fall_membership_school_univ_flag,year
1,01504840100000,03,3,True,True,PK,12,5447.0,5447.0,R,R,2017-01-01
2,01504800100000,03,3,True,True,PK,12,5687.0,5687.0,R,R,2017-01-01
3,01503740100000,03,3,True,True,PK,12,13938.0,13938.0,R,R,2017-01-01
4,01504530100000,03,3,True,True,PK,12,10440.0,10440.0,R,R,2017-01-01
6,01503710100000,03,3,True,True,PK,12,1973.0,1973.0,R,R,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
18671,51502000200000,03,3,True,True,KG,12,2835.0,2835.0,R,R,2017-01-01
18672,51502300200000,03,3,True,True,KG,12,258.0,258.0,R,R,2017-01-01
18673,51502200400000,03,3,True,True,KG,12,1329.0,1329.0,R,R,2017-01-01
18674,51500340100000,07,3,True,True,01,12,NaN,0.0,N,R,2017-01-01


### Expenses & Revenue Schema Tables

#### `melt_df()`
The purpose of this fuction is to convert the data from a wide format to a long format, which is optimal for data normalization in relational databases, and for data visualizations.

In [55]:
def melt_df(df: pd.DataFrame, schema: str, table: str, column_mapping_df: pd.DataFrame, total_columns: list) -> pd.DataFrame:
    # Initialize an empty list for storing column names
    columns_to_use = []
    new_columns = []
    
    # Iterate over each row in the mapping DataFrame
    for _, row in column_mapping_df.iterrows():
        # Check if the table is the specified one or 'all'
        if row['Table'] in [table, 'all']:
            if row['New Name'] not in total_columns:
                # Add the new column name to the list
                columns_to_use.append(row['New Name'])
    
    # Create a new DataFrame with only the selected columns
    new_df = df[columns_to_use].copy()
    
    # Select id_vars for the melt function
    id_vars = ['census_id', 'year'] + [col for col in new_df.columns if col.endswith('_flag')]
    
    # Melt the DataFrame
    if schema == 'expenses':
        new_df = pd.melt(new_df, id_vars=id_vars, var_name='expenditure_title', value_name='amount')
        # The new columns will be 'expenditure_title' and 'amount'
        new_columns = ['expenditure_title', 'amount']
    elif schema == 'revenue':
        new_df = pd.melt(new_df, id_vars=id_vars, var_name='revenue_title', value_name='revenue')
        # The new columns will be 'revenue_title' and 'revenue'
        new_columns = ['revenue_title', 'revenue']
    
    # Ensure the new columns are at indexes 2 and 3
    # Get the list of id_vars that don't include the new columns
    remaining_columns = [col for col in id_vars if col not in new_columns]

    # Reorder columns such that new columns are at index 2 and 3
    ordered_columns = remaining_columns[:2] + new_columns + remaining_columns[2:]
    
    # Reassign the DataFrame with the ordered columns
    new_df = new_df[ordered_columns]
    
    return new_df


#### Create expenditures DF

In [56]:
expenditures = melt_df(df,'expenses', 'expenditures', column_mapping_df, total_columns)
expenditures.head()

,census_id,year,expenditure_title,amount,curr_expenditures_instruction_flag,payments_private_schools_flag,payments_charter_schools_flag,support_services_pupils_flag,support_services_instructional_staff_flag,support_services_general_admin_flag,...,short_term_debt_outstanding_begin_fisc_year_flag,short_term_debt_outstanding_end_fisc_year_flag,assets_sinking_fund_flag,assets_bond_fund_flag,assets_other_funds_flag,utilities_services_flag,tech_related_supplies_services_flag,tech_related_equipment_flag,curr_expenditures_state_local_funds_flag,curr_expenditures_federal_funds_flag
0,01504840100000,2017-01-01,curr_expenditures_instruction,25785000.0,R,R,M,R,R,R,...,M,M,R,R,R,R,M,M,M,M
1,01504800100000,2017-01-01,curr_expenditures_instruction,28419000.0,R,R,M,R,R,R,...,M,M,R,R,R,R,M,M,M,M
2,01503740100000,2017-01-01,curr_expenditures_instruction,90762000.0,R,R,M,R,R,R,...,M,M,R,R,R,R,M,M,M,M
3,01504530100000,2017-01-01,curr_expenditures_instruction,58769000.0,R,R,M,R,R,R,...,M,M,R,R,R,R,M,M,M,M
4,01503710100000,2017-01-01,curr_expenditures_instruction,10378000.0,R,R,M,R,R,R,...,M,M,R,R,R,R,M,M,M,M


#### Create local DataFrame

In [57]:
local = melt_df(df,'revenue', 'local_revenue', column_mapping_df, total_columns)
local.head()

,census_id,year,revenue_title,revenue,parent_government_contributions_flag,propery_taxes_flag,general_sales_tax_flag,public_utility_taxes_flag,individual_corporate_income_tax_flag,all_other_taxes_flag,...,district_activity_receipts_flag,students_fees_nonspecified_flag,other_sales_and_services_flag,rents_and_royalties_flag,sale_of_property_flag,interest_earnings_flag,fines_and_forfeits_flag,private_contributions_local_flag,misc_local_flag,nces_local_and_census_state_rev_flag
0,01504840100000,2017-01-01,parent_government_contributions,NaN,N,R,M,M,M,R,...,R,R,R,R,R,R,R,R,R,R
1,01504800100000,2017-01-01,parent_government_contributions,NaN,N,R,M,M,M,R,...,R,R,R,R,R,R,R,R,R,R
2,01503740100000,2017-01-01,parent_government_contributions,NaN,N,R,M,M,M,R,...,R,R,R,R,R,R,R,R,R,R
3,01504530100000,2017-01-01,parent_government_contributions,NaN,N,R,M,M,M,R,...,R,R,R,R,R,R,R,R,R,R
4,01503710100000,2017-01-01,parent_government_contributions,NaN,N,R,M,M,M,R,...,R,R,R,R,R,R,R,R,R,R


#### Create state DataFrame

In [58]:
state = melt_df(df,'revenue', 'state_revenue', column_mapping_df, total_columns)
state.head()

,census_id,year,revenue_title,revenue,general_formula_assistance_flag,staff_improvement_programs_flag,special_education_programs_flag,compensatory_basic_skills_programs_flag,bilingual_education_state_flag,gifted_talented_programs_flag,vocational_education_programs_flag,school_lunch_programs_flag,capital_outlay_debit_services_programs_flag,transportation_programs_flag,other_programs_state_flag,nonspecified_state_flag,employee_benefits_state_flag,not_employee_benefits_state_flag
0,01504840100000,2017-01-01,general_formula_assistance,26808000.0,R,R,R,R,R,M,M,M,R,R,R,R,M,M
1,01504800100000,2017-01-01,general_formula_assistance,29698000.0,R,R,R,R,R,M,M,M,R,R,R,R,M,M
2,01503740100000,2017-01-01,general_formula_assistance,61809000.0,R,R,R,R,R,M,M,M,R,R,R,R,M,M
3,01504530100000,2017-01-01,general_formula_assistance,47928000.0,R,R,R,R,R,M,M,M,R,R,R,R,M,M
4,01503710100000,2017-01-01,general_formula_assistance,9377000.0,R,R,R,R,R,M,M,M,R,R,R,R,M,M


#### Create federal DataFrame

In [59]:
federal = melt_df(df,'revenue', 'federal_revenue', column_mapping_df, total_columns)
federal.head()

,census_id,year,revenue_title,revenue,title_I_flag,indiv_with_disabilities_flag,math_science_teacher_quality_thru_state_flag,national_activities_for_school_safety_thru_state_flag,voc_tech_education_flag,bilingual_education_flag,other_fed_rev_flag,child_nutrition_act_flag,nonspecified_fed_rev_flag,impact_aid_direct_flag,indian_education_direct_flag,other_direct_fed_rev_flag
0,01504840100000,2017-01-01,title_I_thru_state,2228000.0,R,R,R,R,R,R,R,R,R,R,R,R
1,01504800100000,2017-01-01,title_I_thru_state,2415000.0,R,R,R,R,R,R,R,R,R,R,R,R
2,01503740100000,2017-01-01,title_I_thru_state,1399000.0,R,R,R,R,R,R,R,R,R,R,R,R
3,01504530100000,2017-01-01,title_I_thru_state,749000.0,R,R,R,R,R,R,R,R,R,R,R,R
4,01503710100000,2017-01-01,title_I_thru_state,463000.0,R,R,R,R,R,R,R,R,R,R,R,R


## Create Database Mapping
The keys of the dictionary are the table names within the database.
Values:
- Index 0 = Schema Name
- Index 1 = DataFrame Name

In [60]:
database_map = {'entity' : ['entity', entity],
                'annual_stats' : ['entity', annual_stats],
                'expenditures' : ['expenses', expenditures],
                'federal_revenue' : ['revenue', federal],
                'state_revenue' : ['revenue', state],
                'local_revenue' : ['revenue', local]}

## Database Initialization with Mapped Data
The code snippet enclosed within the conditional block is designed for the initial population of the database. As this project evolves, I will enhance this section with more sophisticated logic and additional functionality to support incremental updates and data management requirements.

The if branches were added to control the use of this cell as I test the code in the notebook. 

In [61]:
use_database = 'n'
if use_database == 'y':
    
    # Read in database credentials from JSON file
    db_credentials_path = os.path.join("..", "LEA_Finance_Survey_DB.json")
    with open(db_credentials_path) as infile:
        credentials = json.load(infile)
    
    # Assign Credentials to Variables
    database_name = credentials['database']
    username = credentials['user']
    password = credentials['password']
    host = credentials['host']
    port = credentials['port']

    # Create a database connection using SQLAlchemy engine
    engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database_name}')
    
    populate_new_tables = 'n' 

    if populate_new_tables == 'y':
        # Iterate over the database_map to insert each DataFrame
        for table_name, [schema_name, df_to_export] in database_map.items():
            df_to_export.to_sql(table_name, engine, schema=schema_name, if_exists='append', index=False)

    engine.dispose()


## Data Normalization for Visualization

In preparing our dataset for visualization in Tableau, I employed normalization techniques on specific columns to ensure that the visualizations are not biased by the scale of the data:

#### Min-Max Scaling
- **Purpose**: To transform the data into a fixed range of 0 to 1, making it easier to visualize different variables on the same scale without distorting the distribution of values. This is particularly important when creating comparative visualizations, such as heatmaps or line charts, where relative scales matter.
- **Applied to**: Columns like `[revenue]`, where we need to maintain the relative distribution of the values for accurate visual comparison.

#### Z-Score Standardization
- **Purpose**: To standardize values so that they have a mean of zero and a standard deviation of one. This normalization is useful for visualizations that compare the relative standing of data points within a distribution, such as histograms or scatter plots.
- **Applied to**: Columns like `[revenue]`, which benefits from showing how many standard deviations away from the mean the data points are, thus facilitating a clear interpretation of outliers and distribution spread.

By normalizing the data before visualization, I aim to create clear and meaningful visualizations that accurately represent the underlying data without the distortion that can come from varying scales.

In [62]:
scaler = MinMaxScaler()

expenditures["amount (Min/Max Scale)"] = scaler.fit_transform(expenditures[["amount"]])
federal["revenue (Min/Max Scale)"] = scaler.fit_transform(federal[["revenue"]])
state["revenue (Min/Max Scale)"] = scaler.fit_transform(state[["revenue"]])
local["revenue (Min/Max Scale)"] = scaler.fit_transform(local[["revenue"]])

In [63]:
scaler = StandardScaler()

expenditures["amount (Z-Score Std)"] = scaler.fit_transform(expenditures[["amount"]])
federal["revenue (Z-Score Std)"] = scaler.fit_transform(federal[["revenue"]])
state["revenue (Z-Score Std)"] = scaler.fit_transform(state[["revenue"]])
local["revenue (Z-Score Std)"] = scaler.fit_transform(local[["revenue"]])

### Code Commentary on Exporting DataFrames

The subsequent code snippet performs the operation of exporting DataFrames to CSV files. These CSV files include derived values such as Z-Scores and Min/Max statistics. This export facilitates further analysis in data visualization tools like Tableau. Although these derived values are excluded from the database for flexibility and to adhere to best practices, they are being included in the CSV exports specifically for the purpose of exploratory analysis outside the database environment.

In [64]:
#entity.to_csv('LEA Finance Survey – Entity Data.csv')
#annual_stats.to_csv('LEA Finance Survey – Entity – Annual Stats Data.csv')
#expenditures.to_csv('LEA Finance Survey – Expenditures Data.csv')
#federal.to_csv('LEA Finance Survey – Federal Revenue Data.csv')
#state.to_csv('LEA Finance Survey – State Revenue Data.csv')
#local.to_csv('LEA Finance Survey – Local Revenue Data.csv')